<a href="https://colab.research.google.com/github/ikonuhov/wbposition/blob/main/wb_position_from_mpstat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выгрузка номера позиции с Wildberries по списку кодов
- Для выгрузки используется сторонний сервис mpstat. 
- Указываем ключ апи от него (можно найти в личном кабинете)
- Загружаем список кодов в формате csv. Первая строка - заголовок
- Указываем откуда брать список кодов
- Скрипт берет код товара вб, обращается к сервису, получает от них список рубрик и текущие позиции за последние 3 дня. Считает среднее.
- Из рубрик выбирает ту, у которой самая низкая средняя позиция, она и будет родительской, основной и оставляет ее в датафрейме
- На выходе формируется excel из собранного датафрейма

In [1]:
import pandas as pd
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import csv
import pprint 
import datetime
key=input()# '605ec9a9a11d77...876269266aef8b021b64e9f46274d83d2aaad624'
pp = pprint.PrettyPrinter(indent=4)

605ec9a9a11d77.876269266aef8b021b64e9f46274d83d2aaad624


In [2]:
# вычисляем период сбора статистики. по умолчанию это последние 3 дня

now = datetime.datetime.utcnow()
start_date = now - datetime.timedelta(days=3)

start_date = start_date.strftime('%Y-%m-%d')
now = now.strftime('%Y-%m-%d')

2022-02-08
2022-02-11


In [3]:
# функция, которая запрашивает данные по апи у мпстата

def request_data_mpstat (url):
  headers = {
      'Content-Type': 'application/json',
      'X-Mpstats-TOKEN': key,
  }

  session = Session()
  session.headers.update(headers)
  try:
    response = session.get(url)
    data = json.loads(response.text)
    
      
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  return (data)

In [4]:
# указываем откуда брать список кодов вб для сбора данных

spisok_sku_csv = input() #"https://raw.githubusercontent.com/ikonuhov/wbposition/main/spisok.csv"
df_sku = pd.read_csv(spisok_sku_csv)

/content/Текущие РК.csv


In [137]:
# формируем пустой датафрейм для дальнейшего заполнения

df = pd.DataFrame(columns =  ['index', 0, 1, 2, 'id', 'level'])

In [138]:
# запукаем сбор данных

for index_sku in range (0, len(df_sku.index)-1):  # создаем цикл от 0 до конца элементов списка
  sku = str(df_sku.iloc[index_sku,0])             # забираем ску 
  url = 'https://mpstats.io/api/wb/get/item/'+sku+'/by_category?d1='+start_date+'&d2='+now    # формируем урл на основе ску, даты начала и даты конца периода
  print(index_sku, url)
  data = request_data_mpstat (url)                # делаем запрос в мпстат
  df1 = pd.DataFrame(data['categories']).T        # формируем промежуточный датафрейм, где сохраним и потом обработаем полученные от мпстата данные
  if df1.empty:                                   # что делаем, если код не найден на мпстат
    df1 = pd.DataFrame([[0, 0, 0]], columns =  [0, 1, 2])
  df1['id'] = sku
  df1 = df1.reset_index()
  if df1['index'].values[0] == 0:
    df1['level'] = 0
  else:
    df1['level'] = df1['index'].str.count('/')    # определяем уровень категории
  df1 = df1.replace({'NaN':0})                    # убираем ошибочные значения
  df1['avg_position'] = df1[[0, 1, 2]].mean(axis=1, skipna=True, numeric_only=True)           # считаем среднюю позицию
  df1 = df1.loc[[df1['avg_position'].idxmin()]]   # оставляем только ту строку, где самый маленький мин
  df = df.append(df1)                             # добавляем полученную строку в общий датафрейм

df

https://mpstats.io/api/wb/get/item/11054284/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/13186733/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/15256077/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/13186738/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686221/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686222/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686224/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686225/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686226/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686228/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686229/by_category?d1=2022-02-08&d2=2022-02-11
https://mpstats.io/api/wb/get/item/17686227/by_category?d1=2022-02-08&d2=202

,index,0,1,2,id,level,avg_position
1,Канцтовары/Чертежные принадлежности/Циркуль,232,229,234,11054284,2,231.666667
1,Канцтовары/Чертежные принадлежности/Циркуль,234,230,233,13186733,2,232.333333
1,Канцтовары/Чертежные принадлежности/Циркуль,291,287,293,15256077,2,290.333333
1,Канцтовары/Чертежные принадлежности/Циркуль,247,240,244,13186738,2,243.666667
2,Канцтовары/Бумажная продукция/Альбом для творч...,2217,2251,2164,17686221,2,2210.666667
...,...,...,...,...,...,...,...
5,Мужчинам/Подарки мужчинам/Другу/Шкатулка,5,3,3,10422713,3,3.666667
7,Мужчинам/Подарки мужчинам/Коллеге/Шкатулка,13,13,13,10422714,3,13.000000
7,Мужчинам/Подарки мужчинам/Коллеге/Шкатулка,69,69,71,10422715,3,69.666667
5,Мужчинам/Подарки мужчинам/Другу/Шкатулка,7,7,7,11512782,3,7.000000


In [139]:
# доукрашаем итоговый датафрейм, округляем, переводим в числа

df['avg_position'] = df['avg_position'].astype(int)
df['id'] = df['id'].astype(int)
df.reset_index(drop = True)

,index,0,1,2,id,level,avg_position
0,Канцтовары/Чертежные принадлежности/Циркуль,232,229,234,11054284,2,231
1,Канцтовары/Чертежные принадлежности/Циркуль,234,230,233,13186733,2,232
2,Канцтовары/Чертежные принадлежности/Циркуль,291,287,293,15256077,2,290
3,Канцтовары/Чертежные принадлежности/Циркуль,247,240,244,13186738,2,243
4,Канцтовары/Бумажная продукция/Альбом для творч...,2217,2251,2164,17686221,2,2210
...,...,...,...,...,...,...,...
747,Мужчинам/Подарки мужчинам/Другу/Шкатулка,5,3,3,10422713,3,3
748,Мужчинам/Подарки мужчинам/Коллеге/Шкатулка,13,13,13,10422714,3,13
749,Мужчинам/Подарки мужчинам/Коллеге/Шкатулка,69,69,71,10422715,3,69
750,Мужчинам/Подарки мужчинам/Другу/Шкатулка,7,7,7,11512782,3,7


In [140]:
# формируем ексельку

df.to_excel('output.xlsx')

# тесты

In [104]:
df

,index,0,1,2,id,level,avg_position,level_0
0,Дом/Досуг и творчество/Творчество и рукоделие/...,903,894,888,21144947,4,895.000000,NaN
0,Дом/Досуг и творчество/Творчество и рукоделие/...,94,0,0,21144943,3,31.333333,NaN
0,Дом/Досуг и творчество/Творчество и рукоделие/...,94,0,0,21144943,3,31.333333,NaN
0,Дом/Досуг и творчество/Творчество и рукоделие/...,129,132,132,42197522,4,131.000000,NaN
2,Дом/Досуг и творчество/Творчество и рукоделие/...,0,80,0,42197523,3,26.666667,NaN
...,...,...,...,...,...,...,...,...
0,0,0,0,0,12638139,0,0.000000,0.0
1,Канцтовары/Чертежные принадлежности/Циркуль,243,234,240,12638141,2,239.000000,NaN
0,0,0,0,0,11054282,0,0.000000,0.0
0,0,0,0,0,12638140,0,0.000000,0.0


In [106]:
df1

,index,0,1,2,id,level,avg_position
1,Канцтовары/Чертежные принадлежности/Циркуль,232,229,234,11054284,2,231.666667


In [59]:
df2 = df1

In [61]:
if df2.empty:
  print ('true')
  df2 = pd.DataFrame([[0, 0, 0, 0, '1231313', 0]], columns =  ['index', 0, 1, 2, 'id', 'level'])
df2

true


,index,0,1,2,id,level
0,0,0,0,0,1231313,0


In [101]:
df2['index'].values[0]

0

In [107]:
sku = '13186733'#str(df_sku.iloc[0,0])
url = 'https://mpstats.io/api/wb/get/item/'+sku+'/by_category?d1='+start_date+'&d2='+now
url

'https://mpstats.io/api/wb/get/item/13186733/by_category?d1=2022-02-08&d2=2022-02-11'

In [108]:
data = request_data_mpstat (url)

https://mpstats.io/api/wb/get/item/13186733/by_category?d1=2022-02-08&d2=2022-02-11


In [109]:
df = pd.DataFrame(data['categories']).T
df['id'] = sku
df = df.reset_index()
df['level'] = df['index'].str.count('/')
df

,index,0,1,2,id,level
0,Канцтовары/Чертежные принадлежности,4614,4608,4558,13186733,1
1,Канцтовары/Чертежные принадлежности/Циркуль,234,230,233,13186733,2


In [110]:
df = df.replace({'NaN':0})
df['avg_position'] = df[[0, 1, 2]].mean (axis=1, skipna = True, numeric_only= True)
df

,index,0,1,2,id,level,avg_position
0,Канцтовары/Чертежные принадлежности,4614,4608,4558,13186733,1,4593.333333
1,Канцтовары/Чертежные принадлежности/Циркуль,234,230,233,13186733,2,232.333333


In [135]:
sku = '12638140'
url = 'https://mpstats.io/api/wb/get/item/'+sku+'/by_category?d1='+start_date+'&d2='+now
data = request_data_mpstat (url)
df1 = pd.DataFrame(data['categories']).T
if df1.empty:
  df1 = pd.DataFrame([[0, 0, 0]], columns =  [0, 1, 2])
df1['id'] = sku
df1 = df1.reset_index()
if df1['index'].values[0] == 0:
  df1['level'] = 0
else:  
  df1['level'] = df1['index'].str.count('/')
df1 = df1.replace({'NaN':0})
df1['avg_position'] = df1[[0, 1, 2]].mean(axis=1, skipna=True, numeric_only=True)
df1 = df1.loc[[df1['avg_position'].idxmin()]]
df = df.append(df1)

https://mpstats.io/api/wb/get/item/12638140/by_category?d1=2022-02-08&d2=2022-02-11


In [136]:
df

,index,0,1,2,id,level,avg_position
0,0,0,0,0,12638140,0,0.000000
1,Канцтовары/Чертежные принадлежности/Циркуль,232,229,234,11054284,2,231.666667
0,0,0,0,0,12638140,0,0.000000


In [132]:
df1

,0,1,2,id
Канцтовары/Чертежные принадлежности,4607,4600,4564,11054284
Канцтовары/Чертежные принадлежности/Циркуль,232,229,234,11054284
